# Analisis exploratorio de datos

In [1]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv('../data/comentarios_mejorados.csv', sep='|')

# Mostrar las filas y columnas del DataFrame
print(df.shape)

# Mostrar las primeras filas del DataFrame
print(df.head())

# Obtener información sobre el DataFrame
print(df.info())

# Obtener estadísticas descriptivas del DataFrame
print(df.describe())

(850, 14)
                  channelId      videoId  \
0  UCmb0LnmFYceH7toqgmUTJDA  U5wTE1t9MqU   
1  UCmb0LnmFYceH7toqgmUTJDA  U5wTE1t9MqU   
2  UCmb0LnmFYceH7toqgmUTJDA  U5wTE1t9MqU   
3  UCmb0LnmFYceH7toqgmUTJDA  U5wTE1t9MqU   
4  UCmb0LnmFYceH7toqgmUTJDA  U5wTE1t9MqU   

                                         textDisplay  \
0       Crack es en ps5 las cinemática o ps4 también   
1  liga femenina , será igual de mala que en la v...   
2                                  Juego malisimo...   
3  A mi me lo regalaron y los graficos justitos p...   
4  Vaya basura de video como se nota que le han p...   

                                        textOriginal      authorDisplayName  \
0       Crack es en ps5 las cinemática o ps4 también  @elreylobodepatio4960   
1  liga femenina , será igual de mala que en la v...             @yemon5329   
2                                  Juego malisimo...       @user-so2tt2wx8n   
3  A mi me lo regalaron y los graficos justitos p...         @sergimateu

## 1. Elimitar columnas que no aportan informacion
En este caso se eliminan las columnas que no aportan informacion al modelo y nos quedamos con el comentario. Eliminamos también los comentarios que son respuestas a otros comentarios basandonos en el hecho de que no aportan información al modelo.

Para ello nos fijamos en el parent_id, si es nulo es un comentario raiz, si no es una respuesta a otro comentario.

In [2]:
# Contamos las filas donde el parent_id sea nulo
print(df.parentId.isnull().sum())
print(df.parentId.isnull().value_counts())

# Filtrar las filas donde el parent_id sea nulo
df = df[df.parentId.isnull()]

# Contamos las filas que quedaron
print(df.shape)


522
parentId
True     522
False    328
Name: count, dtype: int64
(522, 14)


In [3]:
columnas_a_eliminar = ['channelId', 'videoId', 'textOriginal', 'authorDisplayName', 
                       'authorProfileImageUrl', 'authorChannelUrl', 'authorChannelId', 
                       'canRate', 'viewerRating', 'likeCount', 'publishedAt', 
                       'updatedAt', 'parentId']

# Eliminar las columnas
df = df.drop(columnas_a_eliminar, axis=1)

# Mostrar las primeras filas del DataFrame
print(df.head())

                                         textDisplay
0       Crack es en ps5 las cinemática o ps4 también
1  liga femenina , será igual de mala que en la v...
2                                  Juego malisimo...
3  A mi me lo regalaron y los graficos justitos p...
4  Vaya basura de video como se nota que le han p...


## 2. Limpieza de texto
Se eliminan los caracteres especiales, se pasa todo a minusculas y se eliminan espaacios en blanco innecesarios.

In [4]:
import re

# Función para limpiar el texto
def clean_text(text):
    # Eliminar emojis y caracteres especiales manteniendo las tildes y ñ
    text = re.sub(r'([^\s\wñáéíóú]|_)+', '', text)
    
    # Minúsculas
    text = text.lower()
    
    # Eliminar espacios en blanco innecesarios
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Aplicar la función de limpieza de texto a la columna 'textDisplay'
df['textDisplay'] = df['textDisplay'].apply(clean_text)

# Mostrar las primeras filas del DataFrame
print(df.head())

                                         textDisplay
0       crack es en ps5 las cinemática o ps4 también
1  liga femenina será igual de mala que en la vid...
2                                     juego malisimo
3  a mi me lo regalaron y los graficos justitos p...
4  vaya basura de video como se nota que le han p...


## 3. Tokenizacion
Se separa el texto en palabras individuales. Esto nos permite analizar cada palabra por separado y no como una cadena de texto.


In [5]:
import nltk
# Descargar el paquete 'punkt' de NLTK solo es necesario la primera vez
nltk.download('punkt')

# Tokenizar el texto
df['textDisplay'] = df['textDisplay'].apply(nltk.word_tokenize)

print(df.head())


                                         textDisplay
0  [crack, es, en, ps5, las, cinemática, o, ps4, ...
1  [liga, femenina, será, igual, de, mala, que, e...
2                                  [juego, malisimo]
3  [a, mi, me, lo, regalaron, y, los, graficos, j...
4  [vaya, basura, de, video, como, se, nota, que,...


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dylan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 4. Eliminacion de stopwords
Se eliminan las palabras que no aportan informacion al modelo. Estas palabras son las que se encuentran en la lista de stopwords como por ejemplo: "el", "la", "de", etc.

In [6]:
# Eliminar las palabras vacías
from nltk.corpus import stopwords
# Descargar las palabras vacías de NLTK solo es necesario la primera vez
nltk.download('stopwords')
stopwords = stopwords.words('spanish')

# Función para eliminar las palabras vacías
def remove_stopwords(text):
    text = [word for word in text if word not in stopwords]
    return text

# Eliminar las palabras vacías
df['textDisplay'] = df['textDisplay'].apply(remove_stopwords)

print(df.head())

                                         textDisplay
0                      [crack, ps5, cinemática, ps4]
1          [liga, femenina, igual, mala, vida, real]
2                                  [juego, malisimo]
3  [regalaron, graficos, justitos, jugabilidad, m...
4  [vaya, basura, video, nota, pagado, blanquee, ...


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dylan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 5. Stemming o lematizacion
Se eliminan los sufijos de las palabras para quedarnos con la raiz de la palabra. Esto nos permite reducir el numero de palabras que se tienen que analizar. Por ejemplo: "corriendo" se convierte en "correr". 

Esto es útil en análisis de sentimientos ya que la raíz de la palabra puede ser más importante que la palabra en sí. Por ejemplo, "bueno" y "mejor" tienen la misma raíz, "buen", por lo que se puede decir que son similares.

Para lematizar se utiliza la libreria de Spacy, ya que ntltk no tiene soporte para el idioma español.

En nuestro caso instalaremos una versión precompilada de Spacy para el idioma español, ya que la compilación de la libreria puede tardar mucho tiempo.

" pip install https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-2.2.0/es_core_news_sm-2.2.0.tar.gz "

In [8]:
import spacy

# Cargar el modelo de lenguaje español de Spacy
nlp = spacy.load('es_core_news_sm')

# Función para lematizar el texto
def lemmatization(texts):
    output = []
    changed_words_count = 0
    for text in texts:
        # Crear una lista de palabras antes de la lematización
        words_before = text
        
        # Lematizar el texto
        doc = nlp(" ".join(words_before)) 
        words_after = [token.lemma_ for token in doc]
        
        # Comparar las palabras antes y después de la lematización
        for before, after in zip(words_before, words_after):
            if before != after:
                changed_words_count += 1
        
        output.append(words_after)
    
    print(f"Numero de palabras que han cambiado después de la lemantización: {changed_words_count}")
    return output

# Lematizar el texto
df['textDisplay'] = lemmatization(df['textDisplay'])

Number of words changed after lemmatization: 2594


In [ ]:
import spacy

# Cargar el modelo de lenguaje español de Spacy
nlp = spacy.load('es_core_news_sm')

# Lematizar la palabra "corriendo"
doc = nlp("corriendo")
for token in doc:
    print(token.lemma_)